In [3]:
from mlxtend.frequent_patterns import apriori,association_rules

In [4]:
from google.cloud import bigquery
client = bigquery.Client("ford-82e2b9bf5ee42f7db2a764b1")
sql = """
SELECT * FROM `ford-82e2b9bf5ee42f7db2a764b1.team_yabus.final`;
"""

df = client.query(sql).to_dataframe()

Forbidden: 403 POST https://bigquery.googleapis.com/bigquery/v2/projects/ford-82e2b9bf5ee42f7db2a764b1/jobs?prettyPrint=false: Access Denied: Project ford-82e2b9bf5ee42f7db2a764b1: User does not have bigquery.jobs.create permission in project ford-82e2b9bf5ee42f7db2a764b1.

Location: None
Job ID: c3a10751-fc91-423f-aa98-a77a71f5b330


In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
# which of the model has the highest appearences in the dataset
df['model'].value_counts(normalize=True)

In [ ]:
# which of the feature has the highest appearences in the dataset
df['feature_name'].value_counts(normalize=True)

In [ ]:
print(df['feature_name'].nunique())
df['feature_name'].unique()

In [ ]:
print(df['country_code'].nunique())
df['country_code'].unique()

In [ ]:
print(df['assigned_dlr_facing_reg_nm'].nunique())
df['assigned_dlr_facing_reg_nm'].unique()

In [ ]:
print(df['platform'].nunique())
df['platform'].unique()

In [ ]:
df[[
    'make', 'model', 'model_year',
    'feature_usage_count', 'feature_usage_trips','total_trips', 'feature_name','country_code',
    'assigned_dlr_facing_reg_nm', 'contribution_margin', 'platform'
]]

In [ ]:
df1 = df[[
    'vin','make', 'model', 'model_year',
    'feature_usage_count', 'feature_usage_trips','total_trips', 'feature_name','country_code',
    'assigned_dlr_facing_reg_nm', 'contribution_margin', 'platform'
]]

In [ ]:
df1[
    ['feature_usage_count','feature_usage_trips','total_trips','contribution_margin']
] = df1[
    ['feature_usage_count','feature_usage_trips','total_trips','contribution_margin']
].astype(float)

In [ ]:
df1.head()

In [ ]:
df1['model'].unique()

In [ ]:
mybasket = df1[df1['model']=='MUSTANG MACH-E'].groupby(['vin','feature_name'])['feature_usage_trips']\
                                        .sum().unstack().reset_index().fillna(0).set_index('vin')
mybasket

In [ ]:
def hot_encode(x):
    if(x<= 0):
        return 0
    if(x>= 1):
        return 1    

In [ ]:
my_basket_sets = mybasket.applymap(hot_encode)
my_basket_sets

# Training the model

In [ ]:
# generating frequent item sets
my_frequent_items = apriori(my_basket_sets.fillna(0),min_support=0.02,use_colnames=True)
my_frequent_items

In [ ]:
my_rules = association_rules(my_frequent_items,metric='lift')
my_rules

# Creating the function which yields apriori algo and rules

In [ ]:
def apriori_association(model_name:str):
    df1 = df[[
    'vin','make', 'model', 'model_year',
    'feature_usage_count', 'feature_usage_trips','total_trips', 'feature_name','country_code',
    'assigned_dlr_facing_reg_nm', 'contribution_margin', 'platform']]
    df1[
    ['feature_usage_count','feature_usage_trips','total_trips','contribution_margin']
] = df1[
    ['feature_usage_count','feature_usage_trips','total_trips','contribution_margin']
].astype(float)
    mybasket = df1[df1['model']==model_name].groupby(['vin','feature_name'])['contribution_margin']\
                                 .sum().unstack().reset_index().fillna(0).set_index('vin')
    my_basket_sets = mybasket.applymap(hot_encode)
    my_frequent_items = apriori(my_basket_sets,min_support=0.02,use_colnames=True)
    my_rules = association_rules(my_frequent_items.fillna(0),metric='lift')
    return my_frequent_items,my_rules

# F-Series feature association rules

In [ ]:
# F-series
f_frequent_items,f_my_rules = apriori_association(model_name='F-SERIES')

In [ ]:
f_frequent_items

In [ ]:
f_my_rules

# Mustang Mach-E feature association

In [ ]:
mustang_frequent_items,mustang_my_rules = apriori_association('MUSTANG MACH-E')

In [ ]:
mustang_frequent_items

In [ ]:
mustang_my_rules

# Clustering customer w.r.t all the models

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
cust_feature = pd.crosstab(df1['vin'],df1['feature_name'],aggfunc='sum',values=df1['feature_usage_trips'])
cust_feature = cust_feature.fillna(0)

In [ ]:
cust_feature

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_cust_feature = scaler.fit_transform(cust_feature)
pca = PCA(n_components=6)
pca.fit(scaled_cust_feature)
pca_samples = pca.transform(scaled_cust_feature)
ps = pd.DataFrame(pca_samples)
ps.head()

In [ ]:
y = pca.explained_variance_ratio_
plt.bar(x=range(len(y)),height=y)
plt.show()

In [ ]:
# selecting continous variables for clustering
df2 = df[['feature_usage_count', 'feature_usage_trips',
       'total_trips','vehicle_mileage_km','contribution_margin']]

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()
scaled_df = scaler.fit_transform(df2)
scaled_df

In [ ]:
pca = PCA(n_components=3)
pca.fit(scaled_cust_feature)
pca_samples = pca.transform(scaled_cust_feature)
ps = pd.DataFrame(pca_samples)
ps.head()

In [ ]:
ps.shape

In [ ]:
scaled_cust_feature.shape

In [ ]:
from sklearn.cluster import KMeans
#initialize kmeans parameters
kmeans_kwargs = {
"init": "random",
"n_init": 10,
"random_state": 1,
}

#create list to hold SSE values for each k
sse = []
for k in range(1, 8):
    kmeans = KMeans(n_clusters=k, **kmeans_kwargs)
    kmeans.fit(ps)
    sse.append(kmeans.inertia_)

#visualize results
plt.plot(range(1, 8), sse)
plt.xticks(range(1, 8))
plt.xlabel("Number of Clusters")
plt.ylabel("SSE")
plt.show()

In [ ]:
#instantiate the k-means class, using optimal number of clusters
kmeans = KMeans(init="random", n_clusters=4, n_init=10, random_state=1)

#fit k-means algorithm to data
kmeans.fit(ps)

#view cluster assignments for each observation
label = kmeans.labels_

final_df = pd.concat(
    [
        pd.DataFrame(scaled_cust_feature,columns=cust_feature.columns).reset_index(drop=True),
        pd.DataFrame(ps)
    ],axis=1)

final_df.columns.values[-3:] = ['comp1','comp2','comp3']
final_df['label'] = label

In [ ]:
import seaborn as sns
x_axis = final_df['comp2']
y_axis = final_df['comp1']

sns.scatterplot(x=x_axis,y=y_axis,hue=final_df['label'])

In [ ]:
import seaborn as sns
x_axis = final_df['comp2']
y_axis = final_df['comp3']

sns.scatterplot(x=x_axis,y=y_axis,hue=final_df['label'])

In [ ]:
import seaborn as sns
x_axis = final_df['comp1']
y_axis = final_df['comp3']

sns.scatterplot(x=x_axis,y=y_axis,hue=final_df['label'])

# clustering Mustang Mach-E customers

In [ ]:
cust_feature = pd.crosstab(df1[df1['model']=='MUSTANG MACH-E']['vin'],df1[df1['model']=='MUSTANG MACH-E']['feature_name'],
                           aggfunc='sum',values=df1[df1['model']=='MUSTANG MACH-E']['feature_usage_trips'])
cust_feature = cust_feature.fillna(0)
scaler = StandardScaler()
scaled_cust_feature = scaler.fit_transform(cust_feature)
pca = PCA(n_components=6)
pca.fit(scaled_cust_feature)
pca_samples = pca.transform(scaled_cust_feature)
ps = pd.DataFrame(pca_samples)
ps.head()

#initialize kmeans parameters
kmeans_kwargs = {
"init": "random",
"n_init": 10,
"random_state": 1,
}

#create list to hold SSE values for each k
sse = []
for k in range(1, 8):
    kmeans = KMeans(n_clusters=k, **kmeans_kwargs)
    kmeans.fit(ps)
    sse.append(kmeans.inertia_)

#visualize results
plt.plot(range(1, 8), sse)
plt.xticks(range(1, 8))
plt.xlabel("Number of Clusters")
plt.ylabel("SSE")
plt.show()

#instantiate the k-means class, using optimal number of clusters
kmeans = KMeans(init="random", n_clusters=4, n_init=10, random_state=1)

#fit k-means algorithm to data
kmeans.fit(ps)

#view cluster assignments for each observation
label = kmeans.labels_

final_df = pd.concat(
    [
        pd.DataFrame(scaled_cust_feature,columns=cust_feature.columns).reset_index(drop=True),
        pd.DataFrame(ps)
    ],axis=1)

final_df.columns.values[-3:] = ['comp1','comp2','comp3']
final_df['label'] = label

x_axis = final_df['comp2']
y_axis = final_df['comp1']
sns.scatterplot(x=x_axis,y=y_axis,hue=final_df['label'])
plt.show()

x_axis = final_df['comp2']
y_axis = final_df['comp3']
sns.scatterplot(x=x_axis,y=y_axis,hue=final_df['label'])
plt.show()

x_axis = final_df['comp1']
y_axis = final_df['comp3']
sns.scatterplot(x=x_axis,y=y_axis,hue=final_df['label'])
plt.show()

# clustering F-Series customers

In [ ]:
cust_feature = pd.crosstab(df1[df1['model']=='F-SERIES']['vin'],df1[df1['model']=='F-SERIES']['feature_name'],
                           aggfunc='sum',values=df1[df1['model']=='F-SERIES']['feature_usage_trips'])
cust_feature = cust_feature.fillna(0)
scaler = StandardScaler()
scaled_cust_feature = scaler.fit_transform(cust_feature)
pca = PCA(n_components=6)
pca.fit(scaled_cust_feature)
pca_samples = pca.transform(scaled_cust_feature)
ps = pd.DataFrame(pca_samples)
ps.head()

#initialize kmeans parameters
kmeans_kwargs = {
"init": "random",
"n_init": 10,
"random_state": 1,
}

#create list to hold SSE values for each k
sse = []
for k in range(1, 8):
    kmeans = KMeans(n_clusters=k, **kmeans_kwargs)
    kmeans.fit(ps)
    sse.append(kmeans.inertia_)

#visualize results
plt.plot(range(1, 8), sse)
plt.xticks(range(1, 8))
plt.xlabel("Number of Clusters")
plt.ylabel("SSE")
plt.show()

#instantiate the k-means class, using optimal number of clusters
kmeans = KMeans(init="random", n_clusters=4, n_init=10, random_state=1)

#fit k-means algorithm to data
kmeans.fit(ps)

#view cluster assignments for each observation
label = kmeans.labels_

final_df = pd.concat(
    [
        pd.DataFrame(scaled_cust_feature,columns=cust_feature.columns).reset_index(drop=True),
        pd.DataFrame(ps)
    ],axis=1)

final_df.columns.values[-3:] = ['comp1','comp2','comp3']
final_df['label'] = label

x_axis = final_df['comp2']
y_axis = final_df['comp1']
sns.scatterplot(x=x_axis,y=y_axis,hue=final_df['label'])
plt.show()

x_axis = final_df['comp2']
y_axis = final_df['comp3']
sns.scatterplot(x=x_axis,y=y_axis,hue=final_df['label'])
plt.show()

x_axis = final_df['comp1']
y_axis = final_df['comp3']
sns.scatterplot(x=x_axis,y=y_axis,hue=final_df['label'])
plt.show()